In [2]:
## Testing script ##
####################

## without signal, can get stuck!

## in case python package import-ipynb is not installed, comment out next line
## BUT then the solution needs to be in file solutions in plain text
import import_ipynb

import importlib

######################################################################
# initialisations 
######################################################################

testMap = {}
fullnames = {("ALU","A"): "ArrayListWithUndo.append", ("ALU","R"): "ArrayListWithUndo.remove", ("ALU","I"): "ArrayListWithUndo.insert", ("ALU","U"): "ArrayListWithUndo.undo",
            ("NU","A"): "NetworkWithUndo.add", ("NU","M"): "NetworkWithUndo.merge", ("NU","R"): "NetworkWithUndo.root", ("NU","U"): "NetworkWithUndo.undo",
            ("Gt","A"): "Gadget.add", ("Gt","S"): "Gadget.subnets", ("Gt","Ct"): "Gadget.connect", ("Gt","Cn"): "Gadget.clean", ("Gt","U"): "Gadget.undo"}

######################################################################
# code for handling timeouts (for infinite loops)
# main function for running a list of tests
# helper function for converting test results into string
######################################################################
    
def tryWithTimeout(thunk):
    v = mes = None
    try: v = thunk()
    except BaseException as e: mes = f"E: {e}"
    return (v,mes)

def runTests(tests):
    awarded, total, res = 0, 0, ""
    for (UID,test,args) in tests:
        (name,mark,msg,grade) = test(*args) # grade: max; mark: awarded
        s = " "*(2-len(str(UID)))
        s = f"{s}[{UID}]{name}: "
        dots = ''.join(['.' for i in range(50-len(s))])
        res += s+dots
        awarded += grade*mark
        if round(mark,2) == 0:
            res += "error ["+msg+"], awarded: 0 of "+str(grade)
        elif round(mark,2) == 1: 
            res += "success, awarded: "+str(grade)
        else:
            res += "partial success ["+msg+"], awarded: "+str(mark*grade)+" of "+str(grade)
        res += "\n"
        total += grade
    (awarded, total) = (round(awarded,2), round(total,2))
    return (awarded,total,res)

######################################################################
# Parsing and Comparison functions
######################################################################

def parseStack(A):
    s = Stack();
    for i in range(len(A)-1,-1,-1):
        s.push(A[i])
    return s
    
def parseALU(pair): # ALU represented as (arraylist, stack)
    A, B = pair
    ls0 = TestArrayList()
    ls0.appendAll(A)
    ls = ArrayListWithUndo()
    ls.inArray, ls.count = ls0.inArray, ls0.count
    ls.undos = parseStack(B)
    return ls

def parseNU(pair): # NU represented as (ALU, stack)
    A, B = pair    # note parsed NU is unsafe if sum(B) != len(A) 
    ls0 = TestArrayListWithUndo()
    ls0.appendAll(A)
    n = NetworkWithUndo(len(A))
    n.inArray = ls0
    n.undos = parseStack(B)
    return n

def parseGt(triple):  # Gt represented as (array repr, name map, stack)
    A, m, B = triple  # assumes that underlying network is flat
    g = Gadget()
    g.nameMap = m
    n = TestNetworkWithUndo(len(A))
    for (s,j) in A:
        if j<0: g.subsize+=1
        n.dummyRoots[g.nameMap[s]]=j
    g.inNetwork = n
    g.undos = parseStack(B)
    return g

def equalALs(ls1,ls2):
    # returns: 1: equal, -1: unequal array list elements, -2: unequal counts, -3: unequal arrays
    if str(ls1.toArray()) != str(ls2.toArray()): return -1
    if ls1.length() != ls2.length(): return -2
    if str(ls1.inArray) != str(ls1.inArray): return -3
    return 1

def equalStacks(s1,s2):
    if str(s1) == str(s2): return 1
    return -1
    
def equalALUs(ls1,ls2):
    return (equalALs(ls1,ls2),equalStacks(ls1.undos,ls2.undos))

def equalNUs(n1,n2):
    f1 = 1 if str(n1.inArray.toArray())==str(n2.inArray.toArray()) else -1
    f2 = equalStacks(n1.undos,n2.undos)
    return f1,f2

def weakEqualNUs(n1,n2):
    def normal(s):
        A, ptr = [], s.inList
        while ptr!=None:
            A.append(f"{type(ptr[0])}")
            ptr = ptr[1]
        return "#".join(A)
    f1 = 1 if str(n1.inArray.toArray())==str(n2.inArray.toArray()) else -1
    f2 = 1 if normal(n1.undos)==normal(n2.undos) else -1
    return f1,f2

def equalGts(g1,g2):
    f1 = 1 if str(g1.toArray())==str(g2.toArray()) else -1
    f2 = equalStacks(g1.undos,g2.undos)
    return f1,f2

def weakEqualGts(g1,g2):
    def normal(s):
        A, ptr = [], s.inList
        while ptr!=None:
            A.append(f"{ptr[0][0]},{ptr[0][2]}")
            ptr = ptr[1]
        return "#".join(A)
    f1 = 1 if str(g1.toArray())==str(g2.toArray()) else -1
    f2 = 1 if normal(g1.undos)==normal(g2.undos) else -1
    return f1,f2

def equal2DArrs(A1,A2):
    def normal(A):
        for i in range(len(A)): 
            A[i].sort()
            A[i] = ";".join(A[i])
        A.sort()
        return "#".join(A)
    return 1 if normal(A1)==normal(A2) else -1

def markmesOfALUs(f1,f2):     # calculates correctness mark+message given flags f1, f2
    markAL, mesAL = 0.5, ""   # 50% for each of array list and undos stack
    markST, mesST = 0.5, ""
    match f1:
        case 1: pass
        case -1: markAL, mesAL = 0, "wrong array list element"
        case -2: markAL, mesAL = 0.4, "wrong array list count"
        case -3: markAL, mesAL = 0.45, "wrong underlying array"
        case default: assert(0)
    match f2:
        case 1: pass
        case -1: markST, mesST = 0, "wrong undos stack"
        case default: assert(0)
    return (markAL+markST, mesAdd(mesAL,mesST))

def markmesOfNUs(f1,f2):       # calculates correctness mark+message given flags f1, f2
    markALU, mesALU = 0.8, ""  # 80% for array list with undo
    markST, mesST = 0.2, ""    # 20% for undos stack
    match f1:
        case 1: pass
        case -1: markALU, mesALU = 0, "wrong underlying ArrayListWithUndo" 
        case default: assert(0)
    match f2:
        case 1: pass
        case -1: markST, mesST = 0, "wrong undos stack"
        case default: assert(0)
    return (markALU+markST, mesAdd(mesALU,mesST))

def markmesOfGts(f1,f2):       # calculates correctness mark+message given flags f1, f2
    markNU, mesNU = 0.8, ""    # 80% for array representation
    markST, mesST = 0.2, ""    # 20% for undos stack
    match f1:
        case 1: pass
        case -1: markNU, mesNU = 0, "wrong underlying Gadget array" 
        case default: assert(0)
    match f2:
        case 1: pass
        case -1: markST, mesST = 0, "wrong undos stack"
        case default: assert(0)
    return (markNU+markST, mesAdd(mesNU,mesST))

def mesAdd(s1,s2): # add error messages
    if s1 == "": return s2
    if s2 == "": return s1
    return s1+"; "+s2
    
######################################################################
# NetworkWithUndo test functions
######################################################################

# fun: tested function (A,R,I,U)
# nm: test number
# max_mark: max total marks for test
# initl: initial ALU given as (array list elts, undo stack elts)
# args: arguments to test with
# model: model ALU given as (array list elts, undo stack elts)

def ALU_test(fun,nm,max_mark,initl,args,model):
    s = fullnames['ALU',fun]
    testName = f"[{s}{'.'*(25-len(s))}{nm}]"
    initl = parseALU(initl)
    model = parseALU(model)
    match fun:
        case "A": f = appendALUHelper
        case "R": f = removeALUHelper
        case "I": f = insertALUHelper
        case "U": f = undoALUHelper
        case default: assert(0)
    (mark,msg) = f(initl,args,model)
    return (testName,mark,msg,max_mark)

testMap["ALU"] = ALU_test

# Tests appending A on ls vs model, returns mark in [0,1] and a message
#
def appendALUHelper(ls,args,model):
    A = args
    for x in A: 
        _,mes = tryWithTimeout(lambda: ls.append(x)) # TEST THE SOLUTION
        if mes != None: return (0.00,mes)
    f1,f2 = equalALUs(ls,model)
    return markmesOfALUs(f1,f2)

testMap["ALU","A"] = [None]*3

# appendALU tests
testMap["ALU","A"][0] = ("A", "01", 1,
   ([], []),         # start from empty initial ALU 
   [2,3,4,5,5,1,4],  # add these elements, should obtain the following array list and undos stack
   ([2,3,4,5,5,1,4], 
    [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)])
)

testMap["ALU","A"][1] = ("A", "02", 1,
   ([2,3,4,5,5,1,4], 
    [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)]),
   [5,20,7],
   ([2,3,4,5,5,1,4,5,20,7], 
    [('rem', 9, None), ('rem', 8, None), ('rem', 7, None), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)])
)

testMap["ALU","A"][2] = ("A", "03", 1,
   ([2,3,4,5,5,1,4], 
    [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)]),
   [],
   ([2,3,4,5,5,1,4], 
    [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)])
)

# Tests removing A from ls vs model, returns mark in [0,1] and a message
#
def removeALUHelper(ls,args,model):
    A = args
    for x in A: 
        _,mes = tryWithTimeout(lambda: ls.remove(x)) # TEST THE SOLUTION
        if mes!=None: return (0.00,mes)
    f1,f2 = equalALUs(ls,model)
    return markmesOfALUs(f1,f2)

testMap["ALU","R"] = [None]*1
                 
# removeALU tests
testMap["ALU","R"][0] = ("R", "01", 1,
   ([2,3,4,15,5,1,4], 
    [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)]),
   [0,1,2],  # starting from the ALU above, remove these elements, obtain the following ALU
   ([3,15,1,4], 
    [('ins', 2, 5), ('ins', 1, 4), ('ins', 0, 2), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)])
)

# Tests inserting A from ls vs model, returns mark in [0,1] and a message
# 
def insertALUHelper(ls,args,model):
    A = args
    for (i,x) in A: 
        _,mes = tryWithTimeout(lambda: ls.insert(i,x)) # TEST THE SOLUTION
        if mes!=None: return (0.00,mes)    
    f1,f2 = equalALUs(ls,model)
    return markmesOfALUs(f1,f2)

testMap["ALU","I"] = [None]*2

# insertALU tests
testMap["ALU","I"][0] = ("I", "01", 1,
   ([], []),         # start from empty initial ALU 
   [(0,2),(0,3),(1,4),(2,5),(0,5),(5,25),(4,45)],  # add these elements, should obtain the following array list and undos stack
   ([5,3,4,5,45,2,25], 
    [('rem', 4, None), ('rem', 5, None), ('rem', 0, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None), ('rem', 0, None)])
)

testMap["ALU","I"][1] = ("I", "02", 1,
   ([2,3,4,15,5,1,4], 
    [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)]),
   [(0,0),(8,11),(5,2)],  # starting from the ALU above, insert these elements, obtain the following ALU
   ([0,2,3,4,15,2,5,1,4,11], 
    [('rem', 5, None), ('rem', 8, None), ('rem', 0, None), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)])
)

# Tests undoing n times from ls vs model, returns mark in [0,1] and a message
def undoALUHelper(ls,args,model):
    i = args
    for _ in range(i): 
        _,mes = tryWithTimeout(lambda: ls.undo()) # TEST THE SOLUTION
        if mes!=None: return (0.00,mes)
    f1,f2 = equalALUs(ls,model)
    return markmesOfALUs(f1,f2)

testMap["ALU","U"] = [None]*2

# undoALU tests
testMap["ALU","U"][0] = ("U", "01", 1,
   ([], []),         # start from empty initial ALU 
   42,               # undo 42 times, nothing happens 
   ([],[])
)

testMap["ALU","U"][1] = ("U", "02", 1,
   ([2,3,4,15,5,1,4], 
    [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)]),
   3,  # starting from the ALU above, undo 3 times, obtain the following ALU
   ([2,3,4,15], 
    [('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)])
)

######################################################################
# Network test functions
######################################################################

# fun: tested function (A,R,M,U)
# nm: test number
# max_mark: max total marks for test
# initl: initial NU given as (array list elts, undo stack elts)
# args: arguments to test with
# model: model ALU given as (array list elts, undo stack elts)

def NU_test(fun,nm,max_mark,initl,args,model):
    s = fullnames['NU',fun]
    testName = f"[{s}{'.'*(22-len(s))}{nm}]"
    initl = parseNU(initl)
    model = parseNU(model)
    match fun:
        case "A": f = addNUHelper
        case "R": f = rootNUHelper
        case "M": f = mergeNUHelper
        case "U": f = undoNUHelper
        case default: assert(0)
    (mark,msg) = f(initl,args,model)        
    return (testName,mark,msg,max_mark)

testMap["NU"] = NU_test

# Tests adding n elements on n vs model, returns mark in [0,1] and a message
#
def addNUHelper(n,args,model):
    i = args
    for _ in range(i): 
        _,mes = tryWithTimeout(lambda: n.add()) # TEST THE SOLUTION
        if mes!=None: return (0.00,mes)
    f1,f2 = weakEqualNUs(n,model)
    return markmesOfNUs(f1,f2)

testMap["NU","A"] = [None]*2

# addNU tests
testMap["NU","A"][0] = ("A", "01", 1,
   ([], [0]),   # start from empty initial NU 
   5,           # add these many elements, should obtain the following network and undos stack
   ([-1,-1,-1,-1,-1], [1,1,1,1,1,0])
)

testMap["NU","A"][1] = ("A", "02", 1,
   ([-1,-1,-1,-1,-1], [5]), 
   42,
   (47*[-1], 42*[1]+[5])
)

# Tests root(i) in n vs model, returns mark in [0,1] and a message
#
def rootNUHelper(n,args,model):
    A = args
    markRt = 0.4  # 40% to find correct root
    markNU = 0.6  # 60% to apply flat compression 
    mesRt = mesNU = ""
    for (i,j) in A: # TEST THE SOLUTION
        v,mes = tryWithTimeout(lambda: n.root(i))
        if mes!=None: return (0.00,mes)
        if v != j: 
            markRt = 0
            mesRt = "wrong root(s)"
    f1,f2 = weakEqualNUs(n,model)
    x, mesNU = markmesOfNUs(f1,f2)
    return markRt+(x*markNU), mesAdd(mesRt,mesNU)

testMap["NU","R"] = [None]*2

# rootNU tests
testMap["NU","R"][0] = ("R", "01", 1,
   ([8, -2, 1, 0, 0, -1, 8, 8, -6], [9]),
   [(4, 8)],      # starting from the NU above, find this root, obtain the following NU
   ([8, -2, 1, 0, 8, -1, 8, 8, -6], [1, 9] )
)

testMap["NU","R"][1] = ("R", "02", 1,
   ([8, -2, 1, 0, 0, -1, 8, 8, -6], [9]),
   [(8, 8)],      
   ([8, -2, 1, 0, 0, -1, 8, 8, -6], [0, 9] )
)

# Tests merging A in n vs model, returns mark in [0,1] and a message
#
def mergeNUHelper(n,args,model):
    A = args
    for (i,j) in A: 
        _,mes = tryWithTimeout(lambda: n.merge(i,j)) # TEST THE SOLUTION
        if mes!=None: return (0.00,mes)
    f1,f2 = weakEqualNUs(n,model)
    return markmesOfNUs(f1,f2)

testMap["NU","M"] = [None]*3

# mergeNU tests
testMap["NU","M"][0] = ("M", "01", 1,
   ([-1, -1, -1, -1, -1, -1, -1, -1, -1], [9]),
   [(4,0), (0,3), (2,1), (6,8), (8,7)],      # starting from the NU above, merge these clusters, obtain the following NU
   ([-3, -2, 1, 0, 0, -1, 8, 8, -3], [2, 2, 2, 2, 2, 9] )
)

testMap["NU","M"][1] = ("M", "02", 1,
   ([-3, -2, 1, 0, 0, -1, 8, 8, -3], [9]),
   [(0,8)],      
   ([8, -2, 1, 0, 0, -1, 8, 8, -6], [2, 9])
)

testMap["NU","M"][2] = ("M", "03", 1,
   ([-3, -2, 1, 0, 0, -1, 8, 8, -3], [9]),
   [(0,1)],      
   ([-5, 0, 1, 0, 0, -1, 8, 8, -3], [2, 9])
)

# Tests undoing n times from n vs model, returns mark in [0,1] and a message
def undoNUHelper(n,args,model):
    ops, i = args
    for op, x in ops:
        match op:
            case "add": f = lambda: n.add(x)
            case "roo": f = lambda: n.root(x)
            case "mer": f = lambda: n.merge(*x)
            case default: assert(0)
        _,mes = tryWithTimeout(f) # BUILD INITIAL NETWORK
        if mes!=None: return (0,f"undo could not build on add,root,merge [{mes}]")
    for _ in range(i): 
        _,mes = tryWithTimeout(lambda: n.undo()) # TEST THE SOLUTION
        if mes!=None: return (0.00,mes)
    f1,f2 = weakEqualNUs(n,model)
    return markmesOfNUs(f1,f2)

testMap["NU","U"] = [None]*4

# undoNU tests
testMap["NU","U"][0] = ("U", "01", 1,
   ([], []),         # start from empty initial NU 
   ([], 42),         # undo 42 times, nothing happens 
   ([],[])
)

testMap["NU","U"][1] = ("U", "02", 1,
   ([-1]*10, [1,2,3,4]),
   ([], 2),
   ([-1]*7, [3,4])
)

testMap["NU","U"][2] = ("U", "03", 1,
   ([-1]*7, [3,4]),
   ([], 2),
   ([], [])
)

testMap["NU","U"][3] = ("U", "04", 1,
   ([-1]*9, [9]),
   ([("mer",(4,0)), ("mer",(0,3)), ("mer",(2,1)), ("mer",(6,8)), ("mer",(8,7)), ("mer",(0,1))], 
    1),
   ([-3, -2, 1, 0, 0, -1, 8, 8, -3], [2,2,2,2,2,9])
)

######################################################################
# Gadget test functions
######################################################################

# fun: tested function (A,S,Ct,Cn,U)
# nm: test number
# max_mark: max total marks for test
# initl: initial Gt given as (array repr, name map, stack)
# args: arguments to test with
# model: model Gt given as (array repr, name map, stack)

def Gt_test(fun,nm,max_mark,initl,args,model):
    s = fullnames['Gt',fun]
    testName = f"[{s}{'.'*(15-len(s))}{nm}]"    
    initl = parseGt(initl)
    model = parseGt(model)
    match fun:
        case "A": f = addGtHelper
        case "S": f = subnetsGtHelper
        case "Ct": f = connectGtHelper
        case "Cn": f = cleanGtHelper
        case "U": f = undoGtHelper
        case default: assert(0)
    (mark,msg) = f(initl,args,model)       
    return (testName,mark,msg,max_mark)

testMap["Gt"] = Gt_test

# Tests adding n elements on n vs model, returns mark in [0,1] and a message
#
def addGtHelper(g,args,model):
    A = args
    for x in A: 
        _,mes = tryWithTimeout(lambda: g.add(x)) # TEST THE SOLUTION 
        if mes!=None: return (0.00,mes)
    f1,f2 = weakEqualGts(g,model)
    return markmesOfGts(f1,f2)

testMap["Gt","A"] = [None]*2

# addGt tests
testMap["Gt","A"][0] = ("A", "01", 1, ([], {}, []),   # start from empty initial Gt, add following names, get resulting Gt 
   ["128.0.0.1", "216.58.204.68", "212.58.235.1", "qmul", "Nikos.1", "Nikos.2", "Edon.1", "Shitong.1"],           
   ([('128.0.0.1', -1), ('216.58.204.68', -1), ('212.58.235.1', -1), ('qmul', -1), ('Nikos.1', -1), ('Nikos.2', -1), ('Edon.1', -1), ('Shitong.1', -1)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2, 'qmul': 3, 'Nikos.1': 4, 'Nikos.2': 5, 'Edon.1': 6, 'Shitong.1': 7},
    [('rem', 1, 'Shitong.1'), ('rem', 1, 'Edon.1'), ('rem', 1, 'Nikos.2'), ('rem', 1, 'Nikos.1'), ('rem', 1, 'qmul'), ('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)

testMap["Gt","A"][1] = ("A", "02", 1, ([], {}, []),   # start from empty initial Gt, add following names, get resulting Gt 
   ["128.0.0.1", "216.58.204.68", "212.58.235.1", "qmul", "Nikos.1", "Nikos.2", "Edon.1", "Shitong.1", "Nikos.1"],           
   ([('128.0.0.1', -1), ('216.58.204.68', -1), ('212.58.235.1', -1), ('qmul', -1), ('Nikos.1', -1), ('Nikos.2', -1), ('Edon.1', -1), ('Shitong.1', -1)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2, 'qmul': 3, 'Nikos.1': 4, 'Nikos.2': 5, 'Edon.1': 6, 'Shitong.1': 7},
    [('oth', 0, None), ('rem', 1, 'Shitong.1'), ('rem', 1, 'Edon.1'), ('rem', 1, 'Nikos.2'), ('rem', 1, 'Nikos.1'), ('rem', 1, 'qmul'), ('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)

# Tests connecting pairs in A vs model, returns mark in [0,1] and a message
#
def connectGtHelper(g,args,model):
    B, A, con = args
    markCn = 0.1  # 10% to find if connected
    markGt = 0.9  # 90% to connect subnets 
    mesCn = mesGt = ""
    for x in B: 
        _,mes = tryWithTimeout(lambda: g.add(x))
        if mes!=None: return (0,f"connect could not build on add [{mes}]")
    for x,y in A: 
        v,mes = tryWithTimeout(lambda: g.connect(x,y)) # TEST THE SOLUTION 
        if mes!=None: return (0.00,mes)
        if v != con: 
            markCn = 0
            mesCn = "wrong connected return"
    f1,f2 = weakEqualGts(g,model)
    x,mesGt = markmesOfGts(f1,f2)
    return markCn+(x*markGt), mesAdd(mesCn,mesGt)

testMap["Gt","Ct"] = [None]*3

testMap["Gt","Ct"][0] = ("Ct", "01", 1, ([], {}, []),
   (['128.0.0.1', '216.58.204.68', '212.58.235.1', 'qmul', 'Nikos.1', 'Nikos.2', 'Edon.1', 'Shitong.1'],
    [('128.0.0.1', '216.58.204.68'), ('212.58.235.1', 'qmul'), ('Nikos.1', 'Nikos.2'), ('Edon.1', 'Shitong.1')],
    False),
   ([('128.0.0.1', 1), ('216.58.204.68', -2), ('212.58.235.1', 3), ('qmul', -2), ('Nikos.1', 5), ('Nikos.2', -2), ('Edon.1', 7), ('Shitong.1', -2)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2, 'qmul': 3, 'Nikos.1': 4, 'Nikos.2': 5, 'Edon.1': 6, 'Shitong.1': 7},
    [('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('rem', 1, 'Shitong.1'), ('rem', 1, 'Edon.1'), ('rem', 1, 'Nikos.2'), ('rem', 1, 'Nikos.1'), ('rem', 1, 'qmul'), ('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)

testMap["Gt","Ct"][1] = ("Ct", "02", 1, ([], {}, []),
   (['128.0.0.1', '216.58.204.68', '212.58.235.1', 'qmul', 'Nikos.1', 'Nikos.2', 'Edon.1', 'Shitong.1'],
    [('128.0.0.1', '216.58.204.68'), ('212.58.235.1', 'qmul'), ('Nikos.1', 'Nikos.2'), ('Edon.1', 'Shitong.1'), ('216.58.204.68', '212.58.235.1'), ('Nikos.2', 'Edon.1')],
    False),
   ([('128.0.0.1', 3), ('216.58.204.68', 3), ('212.58.235.1', 3), ('qmul', -4), ('Nikos.1', 7), ('Nikos.2', 7), ('Edon.1', 7), ('Shitong.1', -4)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2, 'qmul': 3, 'Nikos.1': 4, 'Nikos.2': 5, 'Edon.1': 6, 'Shitong.1': 7},
    [('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('rem', 1, 'Shitong.1'), ('rem', 1, 'Edon.1'), ('rem', 1, 'Nikos.2'), ('rem', 1, 'Nikos.1'), ('rem', 1, 'qmul'), ('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)

testMap["Gt","Ct"][2] = ("Ct", "03", 1, ([], {}, []),
   (['128.0.0.1', '216.58.204.68', '212.58.235.1', 'qmul', 'Nikos.1', 'Nikos.2', 'Edon.1', 'Shitong.1'],
    [('128.0.0.1', '216.58.204.68'), ('212.58.235.1', 'qmul'), ('Nikos.1', 'Nikos.2'), ('Edon.1', 'Shitong.1'), ('216.58.204.68', '212.58.235.1'), ('Nikos.2', 'Edon.1'), ('qmul', 'Nikos.1')],
    False),
   ([('128.0.0.1', 7), ('216.58.204.68', 7), ('212.58.235.1', 7), ('qmul', 7), ('Nikos.1', 7), ('Nikos.2', 7), ('Edon.1', 7), ('Shitong.1', -8)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2, 'qmul': 3, 'Nikos.1': 4, 'Nikos.2': 5, 'Edon.1': 6, 'Shitong.1': 7},
    [('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('brk', 3, None), ('rem', 1, 'Shitong.1'), ('rem', 1, 'Edon.1'), ('rem', 1, 'Nikos.2'), ('rem', 1, 'Nikos.1'), ('rem', 1, 'qmul'), ('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)

# Tests getting subnets of g n times vs model, returns mark in [0,1] and a message
#
def subnetsGtHelper(g,args,_):
    ops, n, model = args
    for op, x in ops:
        match op:
            case "add": f = lambda: g.add(x)
            case "con": f = lambda: g.connect(*x)
            case default: assert(0)
        _,mes = tryWithTimeout(f) # BUILD INITIAL GADGET
        if mes!=None: return (0,f"subnets could not build on add,connect [{mes}]")
    for _ in range(n): 
        A,mes = tryWithTimeout(lambda: g.subnets()) # TEST THE SOLUTION 
        if mes!=None: return (0.00,mes)
    if type(A)!=type([]): return 0.00, f"wrong subnets type ({type(A)})"
    for x in A: 
        if type(x)!=type([]): return 0.00, f"wrong subnet type ({type(x)})"
    if equal2DArrs(A,model) != 1: return 0.00,"wrong subnets"
    return 1.00,""

testMap["Gt","S"] = [None]*3

testMap["Gt","S"][0] = ("S", "01", 1, ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1')],
    1,
    [['128.0.0.1'], ['216.58.204.68'], ['212.58.235.1'], ['qmul'], ['Nikos.1'], ['Nikos.2'], ['Edon.1'], ['Shitong.1']]
   ), ([], {}, [])
)

testMap["Gt","S"][1] = ("S", "02", 1, ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1')],
    1,
    [['216.58.204.68'], ['128.0.0.1'], ['212.58.235.1'], ['qmul'], ['Nikos.2'], ['Nikos.1'], ['Edon.1'], ['Shitong.1']]
   ), ([], {}, [])
)

testMap["Gt","S"][2] = ("S", "03", 1, ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1'), ("con",('Nikos.1','Nikos.2')), ("con",('Nikos.1','Edon.1'))],
    1,
    [['216.58.204.68'], ['128.0.0.1'], ['212.58.235.1'], ['qmul'], ['Nikos.2', 'Nikos.1', 'Edon.1'], ['Shitong.1']]
   ), ([], {}, [])
)

# Tests cleaning names in A from g vs model and Us array and returns mark in [0,1] and a message
def cleanGtHelper(g,args,model):
    ops, A = args
    for op, x in ops:
        match op:
            case "add": f = lambda: g.add(x)
            case "con": f = lambda: g.connect(*x)
            case default: assert(0)
        _,mes = tryWithTimeout(f) # BUILD INITIAL GADGET
        if mes!=None: return (0,f"clean could not build on add,connect [{mes}]")
    for s in A: 
        _,mes = tryWithTimeout(lambda: g.clean(s))  # TEST THE SOLUTION
        if mes!=None: return (0.00,mes)
    f1,f2 = weakEqualGts(g,model)
    return markmesOfGts(f1,f2)

testMap["Gt","Cn"] = [None]*3

testMap["Gt","Cn"][0] = ("Cn", "01", 1,
   ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1')],
    ['qmul']),
   ([('128.0.0.1', -1), ('216.58.204.68', -1), ('212.58.235.1', -1)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2},
    [('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)

testMap["Gt","Cn"][1] = ("Cn", "02", 1,
   ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1')],
    ['qmul', '128.0.0.1']),
   ([], {}, [])
)

testMap["Gt","Cn"][2] = ("Cn", "03", 1,
   ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("con",("qmul","Nikos.1")), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1')],
    ['Nikos.1']),
   ([('128.0.0.1', -1), ('216.58.204.68', -1), ('212.58.235.1', -1), ('qmul', -1)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2, 'qmul': 3},
    [('rem', 1, 'qmul'), ('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)


# Tests undoing an array A of times from g vs model and returns mark in [0,1] and a message
def undoGtHelper(g,args,model):
    ops, A = args
    for op, x in ops:
        match op:
            case "add": f = lambda: g.add(x)
            case "con": f = lambda: g.connect(*x)
            case default: assert(0)
        _,mes = tryWithTimeout(f) # BUILD INITIAL GADGET
        if mes!=None: return (0,f"undo could not build on add,connect [{mes}]")
    for i in A: 
        _,mes = tryWithTimeout(lambda: g.undo(i))  # TEST THE SOLUTION
        if mes!=None: return (0.00,mes)
    f1,f2 = weakEqualGts(g,model)
    return markmesOfGts(f1,f2)

testMap["Gt","U"] = [None]*5

# undoGt tests
testMap["Gt","U"][0] = ("U", "01", 1,
   ([], {}, []),   # start from empty initial Gt, undo, nothing happens 
   ([], [21, 21]), 
   ([], {}, [])
)

testMap["Gt","U"][1] = ("U", "02", 1,
   ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1')],
    [4]),
   ([('128.0.0.1', -1), ('216.58.204.68', -1), ('212.58.235.1', -1), ('qmul', -1)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2, 'qmul': 3},
    [('rem', 1, 'qmul'), ('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)

testMap["Gt","U"][2] = ("U", "03", 1,
   ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1')],
    [4, 4]),
   ([], {}, [])
)

testMap["Gt","U"][3] = ("U", "04", 1,
   ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("con",("qmul","Nikos.1")), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1')],
    [3]),
   ([('128.0.0.1', -1), ('216.58.204.68', -1), ('212.58.235.1', -1), ('qmul', 4), ('Nikos.1', -2)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2, 'qmul': 3, 'Nikos.1': 4},
    [('brk', 3, None), ('rem', 1, 'Nikos.1'), ('rem', 1, 'qmul'), ('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)

testMap["Gt","U"][4] = ("U", "05", 1,
   ([], {}, []),
   ([("add",'128.0.0.1'), ("add",'216.58.204.68'), ("add",'212.58.235.1'), ("add",'qmul'), ("add",'Nikos.1'), ("con",("qmul","Nikos.1")), ("add",'Nikos.2'), ("add",'Edon.1'), ("add",'Shitong.1')],
    [3, 1]),
   ([('128.0.0.1', -1), ('216.58.204.68', -1), ('212.58.235.1', -1), ('qmul', -1), ('Nikos.1', -1)],
    {'128.0.0.1': 0, '216.58.204.68': 1, '212.58.235.1': 2, 'qmul': 3, 'Nikos.1': 4},
    [('rem', 1, 'Nikos.1'), ('rem', 1, 'qmul'), ('rem', 1, '212.58.235.1'), ('rem', 1, '216.58.204.68'), ('rem', 1, '128.0.0.1')] 
   )
)


######################################################################
# Gather all tests 
######################################################################

def runAllTests():
    vs  = [0,0,""]
    n = 0
    ranges = list(zip(["ALU"]*4,["A","R","I","U"]))+list(zip(["NU"]*4,["A","M","R","U"]))+list(zip(["Gt"]*5,["A","S","Ct","Cn","U"]))

    for x,y in ranges:
        f, tests = testMap[x], testMap[x,y]
        v = runTests(list(zip(range(n,n+len(tests)),[f]*len(tests),tests)))    
        for i in range(3): vs[i] += v[i]
        s = fullnames[x,y]
        testMap[x,y] = (tests, v, (f"\n -> {s}{'.'*(25-len(s))} : {v[0]} [{v[1]}]"))
        n+=len(tests)
    
    results =  f"Testing ...\n{vs[2]}"
    for x,y in ranges:
        results += testMap[x,y][2]
    results += f"\n-> *Total: {round(vs[0],2)} [{vs[1]}]"
    print(results)
#     return (sid,cntM,maxM,xM,remM,updM,marks,results)

######################################################################
# main test script (full, marks add up to ??)
######################################################################

# import student code
module = "solutions" # you solutions file here!

# import classes
sid = 0
flag = ""
try:
    mod = importlib.import_module(module.split(".")[0])
    ArrayListWithUndo = mod.ArrayListWithUndo
    Stack = mod.Stack
    NetworkWithUndo = mod.NetworkWithUndo
    Gadget = mod.Gadget
except Exception as err: 
    flag = "IMPORT: "+str(err)

# run tests
if flag != "":
    s = "\nTotal testing marks [80]: 0\nError: "+flag
    print(s)
    res = (sid,0,0,0,0,0,s)
else:
    print("Imported and will test")
    res = runAllTests()

Imported and will test
Testing ...
 [0][ArrayListWithUndo.append.01]: ...............success, awarded: 1
 [1][ArrayListWithUndo.append.02]: ...............success, awarded: 1
 [2][ArrayListWithUndo.append.03]: ...............success, awarded: 1
 [3][ArrayListWithUndo.remove.01]: ...............success, awarded: 1
 [4][ArrayListWithUndo.insert.01]: ...............success, awarded: 1
 [5][ArrayListWithUndo.insert.02]: ...............success, awarded: 1
 [6][ArrayListWithUndo.undo...01]: ...............success, awarded: 1
 [7][ArrayListWithUndo.undo...02]: ...............success, awarded: 1
 [8][NetworkWithUndo.add...01]: ..................success, awarded: 1
 [9][NetworkWithUndo.add...02]: ..................success, awarded: 1
[10][NetworkWithUndo.merge.01]: ..................success, awarded: 1
[11][NetworkWithUndo.merge.02]: ..................success, awarded: 1
[12][NetworkWithUndo.merge.03]: ..................success, awarded: 1
[13][NetworkWithUndo.root..01]: ..................succe

In [1]:
# Helper classes for testing #
##############################

class TestArrayList:
    def __init__(self):
        self.inArray = [0 for i in range(10)]
        self.count = 0
        
    def appendAll(self, A):
        for x in A: self.append(x)
        
    def get(self, i):
        return self.inArray[i]

    def set(self, i, e):
        self.inArray[i] = e

    def length(self):
        return self.count

    def append(self, e):
        self.inArray[self.count] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()

    def insert(self, i, e):
        for j in range(self.count,i,-1):
            self.inArray[j] = self.inArray[j-1]
        self.inArray[i] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()
    
    def remove(self, i):
        self.count -= 1
        val = self.inArray[i]
        for j in range(i,self.count):
            self.inArray[j] = self.inArray[j+1]
        return val

    def _resizeUp(self):
        newArray = [0 for i in range(2*len(self.inArray))]
        for j in range(len(self.inArray)):
            newArray[j] = self.inArray[j]
        self.inArray = newArray
        
    def toArray(self):
        return self.inArray[:self.count]

    def __str__(self):
        if self.count == 0: return "[]"
        s = "["
        for i in range(self.count-1): s += str(self.inArray[i])+", "
        return s+str(self.inArray[self.count-1])+"]" 
    
class TestArrayListWithUndo(TestArrayList):
    def __init__(self):
        super().__init__()
        self.oldList = None
            
    def set(self, i, v):
        self._copyToOld()
        super().set(i,v)

    def append(self, v):
        self._copyToOld()
        super().append(v)
        
    def insert(self, i, v):
        self._copyToOld()
        super().insert(i,v)
    
    def remove(self, i):
        self._copyToOld()
        super().remove(i)
    
    def undo(self):
        if self.oldList == None: return
        temp = self.oldList.oldList
        self.inArray, self.count = self.oldList.inArray, self.oldList.count
        self.oldList = temp
        
    def _copyToOld(self):
        temp = TestArrayListWithUndo()
        temp.inArray, temp.count, temp.oldList = self.inArray[:], self.count, self.oldList
        self.oldList = temp        

    def __str__(self):
        # prints just the array list
        return str(self.toArray())
    
class TestNetworkWithUndo:
    def __init__(self, N):
        self.dummyRoots = {}
        for i in range(N): self.dummyRoots[i]=i
        self.oldNet = None
    
    def getSize(self):
        return len(self.dummyRoots)
        
    def add(self):
        self._copyToOld()
        N = self.getSize()
        self.dummyRoots[N]=N 
    
    def root(self, i):
        self._copyToOld()
        return self.dummyRoots[i]
    
    def _getSize(self, i):
        size_i = 0
        for k in self.dummyRoots:
            if self.dummyRoots[k]==i: size_i += 1
        return size_i
            
    def merge(self, i, j):
        if i!=self.dummyRoots[i] or j!=self.dummyRoots[j]: assert(0)
        self._copyToOld()
        if i==j: return
        size_i, size_j = self._getSize(i), self._getSize(j)
        if size_i<=size_j:
            for k in self.dummyRoots:
                if self.dummyRoots[k]==i: self.dummyRoots[k]=j
        else: 
            for k in self.dummyRoots:
                if self.dummyRoots[k]==j: self.dummyRoots[k]=i
        
    def undo(self):
        if self.oldNet == None: return
        self.dummyRoots, self.oldNet = self.oldNet.dummyRoots, self.oldNet.oldNet

    def _copyToOld(self):
        temp = TestNetworkWithUndo(0)
        temp.dummyRoots, temp.oldNet = self.dummyRoots.copy(), self.oldNet
        self.oldNet = temp                
           
    def toArray(self):
        A = [self.dummyRoots[i] for i in range(len(self.dummyRoots))]
        for i in range(len(A)):
            if A[i]==i: A[i] = -self._getSize(i)
        return A
            
    def __str__(self):
        return str(self.toArray())